In [1]:
from langchain_community.document_loaders.pdf import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [4]:
caminhos =[
    "../arquivos/Explorando o Universo das IAs com Hugging Face.pdf",
    "../arquivos/Apostila LangChain.pdf",
    "../arquivos/Explorando a API da OpenAI.pdf",
    "../arquivos/Explorando a API da OpenAI.pdf"
]

paginas = []

for caminho in caminhos:
    loader = PyPDFLoader(caminho)
    paginas.extend(loader.load())
    
recur_split= RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
    separators=["\n\n", "\n", " ", ""]
)

documents = recur_split.split_documents(paginas)

In [5]:
for i, doc in enumerate(documents):
    doc.metadata["source"] = doc.metadata["source"].replace("../arquivos/", "")
    doc.metadata["doc_id"] = i

In [6]:
documents[0].metadata

{'source': 'Explorando o Universo das IAs com Hugging Face.pdf',
 'page': 0,
 'doc_id': 0}

In [7]:
__import__('pysqlite3')
import sys
sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma

embedding_model = OpenAIEmbeddings()

In [9]:
diretorio = '../arquivos/chroma_retrieval_bd'

vectordb = Chroma.from_documents(
    documents = documents,
    embedding=embedding_model,
    persist_directory=diretorio
)

In [11]:
pergunta = "O que é o OpenAI?"

docs = vectordb.similarity_search(pergunta, k= 3)

for doc in docs:
    print(doc.page_content)
    print(f"======================{doc.metadata}", "\n")

Explorando a API da OpenAI
01. Bem-vindos ao curso Explorando a API da OpenAI
Olá e bem-vindos ao nosso curso “Explorando a API da OpenAI”! Estou super animado por ter vocês
aqui e mal posso esperar para explorarmos juntos esse universo fascinante da programação e da
inteligência artificial.
Vocês já devem ter percebido que a IA está mudando o jogo em muitos campos, certo? É incrível como
ela está transformando a maneira como lidamos com informações e realiza tarefas que antes pareciam
======================{'doc_id': 558, 'page': 4, 'source': 'Explorando a API da OpenAI.pdf'} 

Explorando a API da OpenAI
01. Bem-vindos ao curso Explorando a API da OpenAI
Olá e bem-vindos ao nosso curso “Explorando a API da OpenAI”! Estou super animado por ter vocês
aqui e mal posso esperar para explorarmos juntos esse universo fascinante da programação e da
inteligência artificial.
Vocês já devem ter percebido que a IA está mudando o jogo em muitos campos, certo? É incrível como
ela está transformando